In [ ]:
%pip install pandas
%pip install boto3
%pip install psycopg2
%pip install sqlalchemy
%pip install dotenv


%pip matplotlib
%pip plotly
%pip seaborn 


# Run this cell to install these packages
# !pip install pandas boto3 sqlalchemy python-dotenv

In [ ]:
# Import dependencies
import boto3
import os
import pandas as pd
from sqlalchemy import create_engine, text
import psycopg2
from io import StringIO

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import plotly.express as px
import plotly.graph_objects as go


from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors


#pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,  StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer


#  Model Comparison 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, confusion_matrix


# --- Regression Models ---
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


from imblearn.over_sampling import SMOTE


from sklearn.model_selection import GridSearchCV

import shap

import joblib


sns.set(style="whitegrid")

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# Load environment variables from .env file
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# <h2 align="center">Data Input</h2>


### Define a Function to Query & Display Results

In [ ]:
# Create a function to fetch data from the database
def get_db_connection():
    db_url = (
        f"postgresql://{os.environ['DB_USER']}:{os.environ['DB_PASSWORD']}@"
        f"{os.environ['DB_HOST']}:{os.environ['DB_PORT']}/{os.environ['DB_NAME']}"
    )
    return create_engine(db_url)

# Instantiate the database connection
engine = get_db_connection()


# Define query to fetch data from each table
query_clinics = "SELECT * FROM clinics;"
query_patients = "SELECT * FROM patients;"
query_sessions = "SELECT * FROM sessions;"
query_feedback = "SELECT * FROM feedback;"
query_dropout_flags = "SELECT * FROM dropout_flags;"
query_interventions = "SELECT * FROM interventions;"

# Load data from each table into a DataFrame
clinics_df = pd.read_sql(query_clinics, engine)
patients_df = pd.read_sql(query_patients, engine)
sessions_df = pd.read_sql(query_sessions, engine)
feedback_df = pd.read_sql(query_feedback, engine)
dropout_flags_df = pd.read_sql(query_dropout_flags, engine)
interventions_df = pd.read_sql(query_interventions, engine)

In [ ]:
# 1. EDA: Data Overview

print("clinics_df:", clinics_df.shape)
print(clinics_df.head())
print("patients_df:", patients_df.shape)
print(patients_df.head())
print("sessions_df:", sessions_df.shape)
print(sessions_df.head())
print("feedback_df:", feedback_df.shape)
print(feedback_df.head())
print("dropout_flags_df:", dropout_flags_df.shape)
print(dropout_flags_df.head())
print("interventions_df:", interventions_df.shape)
print(interventions_df.head())

In [ ]:
# --- 1. Data Overview ---
print("clinics_df:", clinics_df.shape)
print("patients_df:", patients_df.shape)
print("sessions_df:", sessions_df.shape)
print("feedback_df:", feedback_df.shape)
print("dropout_flags_df:", dropout_flags_df.shape)
print("interventions_df:", interventions_df.shape)

### Feature Engineering
- Create New features
- Merge relevant datasets

## NOTE: 
After the EDA, the data is pretty messy, hence we will:
- define the `patients data and session data` and merge the data together followed by, 
- creating a pipeline (define numerical and categorical columns),
- Then preprocessing the data before doing the clustering,
- Then do the clustering,

In [ ]:
# Session Aggregation (Dynamic Features)
sess_agg = (
    sessions_df
    .sort_values(['patient_id', 'date'])
    .assign(
        pain_delta=lambda d: d.groupby('patient_id')['pain_level'].diff(),
        session_gap=lambda d: d.groupby('patient_id')['date'].diff().dt.days
    )
    .groupby('patient_id')
    .agg(
        n_sessions=('session_id', 'count'),
        avg_session_duration=('duration', 'mean'),
        first_week=('week', 'min'),
        last_week=('week', 'max'),
        mean_pain=('pain_level', 'mean'),
        mean_pain_delta=('pain_delta', 'mean'),
        max_pain_delta=('pain_delta', 'max'),
        min_pain_delta=('pain_delta', 'min'),
        std_pain_delta=('pain_delta', 'std'),
        home_adherence_mean=('home_adherence_pc', 'mean'),
        home_adherence_std=('home_adherence_pc', 'std'),
        satisfaction_mean=('satisfaction', 'mean'),
        avg_session_gap=('session_gap', 'mean'),
        missed_sessions=('session_gap', lambda x: (x > 7).sum())
    )
)
print("sess_agg shape:", sess_agg.shape)
print(sess_agg.head())

In [ ]:
sess_agg.columns

In [ ]:
sess_agg.isnull().sum()

In [ ]:
# Create adherence_class from home_adherence_mean in sess_agg, then drop home_adherence_mean

# Bin as percentage
bins = [-1, 40, 60, np.inf]  # <60%: Low, 60-85%: Medium, >85%: High
labels = ['Low', 'Medium', 'High']
sess_agg['adherence_class'] = pd.cut(sess_agg['home_adherence_mean'], bins=bins, labels=labels)

In [ ]:
sess_agg.head(2)

In [ ]:
sess_agg.isnull()

In [ ]:
print("Adherence class distribution:")
print(sess_agg['adherence_class'].value_counts())

In [ ]:
# 2. Aggregate feedback data (sentiment, if available)
if 'sentiment' in feedback_df.columns:
    feedback_agg = (
        feedback_df
        .merge(sessions_df[['session_id', 'patient_id']], on='session_id', how='left')
        .groupby('patient_id')
        .agg(
            feedback_sentiment_mean=('sentiment', 'mean'),
            feedback_sentiment_std=('sentiment', 'std'),
            feedback_count=('sentiment', 'count')
        )
    )
else:
    feedback_agg = pd.DataFrame()
print("feedback_agg shape:", feedback_agg.shape)
print(feedback_agg.head())

In [ ]:
# 3. Aggregate interventions (count and type)
intervention_agg = (
    interventions_df
    .groupby('patient_id')
    .agg(
        n_interventions=('intervention_id', 'count'),
        unique_interventions=('channel', pd.Series.nunique)
    )
)
print("intervention_agg shape:", intervention_agg.shape)
print(intervention_agg.head())

In [ ]:
# Merge All Features
patient_static = patients_df.set_index('patient_id')
features = patient_static.join(sess_agg, how='left')
if not feedback_agg.empty:
    features = features.join(feedback_agg, how='left')
features = features.join(intervention_agg, how='left')

print("Final feature set shape:", features.shape)
print(features.head())

In [ ]:
features.head()

In [ ]:
# 3. Feature Selection & Preprocessing

# --- Select Features ---
num_cols = [
    "age", "bmi", "n_sessions", "avg_session_duration", "mean_pain", "mean_pain_delta",
    "max_pain_delta", "min_pain_delta", "std_pain_delta", "home_adherence_mean",
    "home_adherence_std", "satisfaction_mean", "avg_session_gap", "missed_sessions",
    "feedback_sentiment_mean", "feedback_sentiment_std", "feedback_count",
    "n_interventions", "unique_interventions"
]
cat_cols = [
    "gender", "smoker", "chronic_cond", "injury_type", "referral_source", "insurance_type"
]
target_col = "home_adherence_mean"

# --- Data Cleaning ---
features[num_cols] = features[num_cols].apply(pd.to_numeric, errors='coerce')
features[cat_cols] = features[cat_cols].astype("string")
data = features.dropna(subset=[target_col])

print("Modeling data shape:", data.shape)
print(data.head())

In [ ]:
data.head()

In [ ]:
# Fix ambiguous NA in 'smoker' column (and any other string/categorical columns)
# Replace pd.NA/NA/None with string 'Unknown' before modeling or any boolean operation

for col in cat_cols:
    data[col] = data[col].astype("string").fillna("Unknown").replace({pd.NA: "Unknown", None: "Unknown"})

# Now you can safely use boolean operations or fit models without TypeError
data['smoker'].unique()

### REGRESSSION MODEL

In [ ]:
# Regression: Predict Adherence Level (Continuous)

X = data[num_cols + cat_cols]
y = data[target_col]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

In [ ]:
# --- Preprocessing Pipeline ---
numeric_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", RobustScaler())
])
categorical_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])
preprocessor = ColumnTransformer([
    ("num", numeric_pipe, num_cols),
    ("cat", categorical_pipe, cat_cols)
])

In [ ]:
regressors = {
    "Linear Regression": LinearRegression(),
    "Ridge": Ridge(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42, verbosity=0),
    "CatBoost": CatBoostRegressor(verbose=0, random_state=42),
    "MLPRegressor": MLPRegressor(max_iter=500, random_state=42)
}

for name, model in regressors.items():
    pipe = Pipeline([
        ("preprocessor", preprocessor),
        ("model", model)
    ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    print(f"\n{name}")
    print("MAE:", mean_absolute_error(y_test, y_pred))
    mse = mean_squared_error(y_test, y_pred)
    rmse = mse ** 0.5
    print("RMSE:", rmse)
    print("R2:", r2_score(y_test, y_pred))

In [ ]:
# Hyperparameter Tuning Example (Random Forest)
param_grid = {
    "model__n_estimators": [100, 200],
    "model__max_depth": [None, 5, 10]
}
rf_pipe = Pipeline([
    ("preprocessor", preprocessor),
    ("model", RandomForestRegressor(random_state=42))
])
grid = GridSearchCV(rf_pipe, param_grid, scoring='neg_mean_absolute_error', cv=5)
grid.fit(X_train, y_train)
print("\nBest Random Forest Params:", grid.best_params_)
best_rf = grid.best_estimator_
y_pred = best_rf.predict(X_test)
print("Tuned Random Forest MAE:", mean_absolute_error(y_test, y_pred))
rmse = mse ** 0.5
print("RMSE:", rmse)
print("Tuned Random Forest R2:", r2_score(y_test, y_pred))

In [ ]:
# Plot predictions vs actual for regression models
for name, model in regressors.items():
    pipe = Pipeline([
        ("preprocessor", preprocessor),
        ("model", model)
    ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    plt.figure(figsize=(6, 6))
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.xlabel("Actual home_adherence_mean")
    plt.ylabel("Predicted home_adherence_mean")
    plt.title(f"{name}: Actual vs Predicted")
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.show()

In [ ]:
# Save tuned random forest regression model
joblib.dump(best_rf, "models/saved_models/best_random_forest_regressor.joblib")

# SHAP plot for regressor
try:
    explainer = shap.Explainer(best_rf.named_steps["model"], best_rf.named_steps["preprocessor"].transform(X_test))
    shap_values = explainer(best_rf.named_steps["preprocessor"].transform(X_test))
    shap.summary_plot(shap_values, feature_names=best_rf.named_steps["preprocessor"].get_feature_names_out(), show=False)
    plt.title("SHAP Summary: Random Forest Regressor")
    plt.savefig("reports/figures/shap_rf_regressor.png")
    plt.close()
except Exception as e:
    print(f"SHAP failed: {e}")

### Classification

In [ ]:
features.shape

In [ ]:
features.head()

In [ ]:
# Drop the home_adherence_mean column
features = features.drop(columns=['home_adherence_mean'])

features.head(2)

In [ ]:
features.shape

In [ ]:
#  Feature Selection & Preprocessing

# Select Features 
num_cols = [
    "age", "bmi", "n_sessions", "avg_session_duration", "mean_pain", "mean_pain_delta",
    "max_pain_delta", "min_pain_delta", "std_pain_delta",
    "home_adherence_std", "satisfaction_mean", "avg_session_gap", "missed_sessions",
    "feedback_sentiment_mean", "feedback_sentiment_std", "feedback_count",
    "n_interventions", "unique_interventions"
]
cat_cols = [
    "gender", "smoker", "chronic_cond", "injury_type", "referral_source", "insurance_type"
]
target_col = "adherence_class"

# --- Data Cleaning ---
features[num_cols] = features[num_cols].apply(pd.to_numeric, errors='coerce')
features[cat_cols] = features[cat_cols].astype("string")
data = features.dropna(subset=[target_col])

print("Modeling data shape:", data.shape)
print(data.head())

In [ ]:
# --- Data Cleaning ---
features[num_cols] = features[num_cols].apply(pd.to_numeric, errors='coerce')
features[cat_cols] = features[cat_cols].astype("string")
data = features.dropna(subset=[target_col])

# Replace ambiguous NA in categorical columns with 'Unknown'
for col in cat_cols:
    data[col] = data[col].astype("string").fillna("Unknown").replace({pd.NA: "Unknown", None: "Unknown"})

# Fill numeric columns with median to avoid NaN
for col in num_cols:
    data[col] = data[col].fillna(data[col].median())

In [ ]:
# Feature Selection using RandomForest for importance ranking
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# Encode categorical columns for feature selection
data_fs = data.copy()
for col in cat_cols:
    le = LabelEncoder()
    data_fs[col] = le.fit_transform(data_fs[col].astype(str))

# Encode target
le_target = LabelEncoder()
y_fs = le_target.fit_transform(data_fs[target_col])

In [ ]:
# Fit RandomForest to get feature importances
rf_fs = RandomForestClassifier(n_estimators=100, random_state=42)
rf_fs.fit(data_fs[num_cols + cat_cols], y_fs)
importances = rf_fs.feature_importances_

# Select top features (e.g., top 10)
import numpy as np
feature_names = num_cols + cat_cols
top_idx = np.argsort(importances)[::-1][:10]
selected_features = [feature_names[i] for i in top_idx]
print("Top features for classification:", selected_features)

In [ ]:
# Prepare data for classification
X_cls = data[selected_features]
y_cls = data[target_col]

# Encode categorical columns in X_cls for SMOTE
X_cls_enc = X_cls.copy()
for col in X_cls_enc.select_dtypes(include=["object", "string", "category"]).columns:
    le = LabelEncoder()
    X_cls_enc[col] = le.fit_transform(X_cls_enc[col].astype(str))

In [ ]:
# Split data
from sklearn.model_selection import train_test_split
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(
    X_cls_enc, y_cls, stratify=y_cls, test_size=0.2, random_state=42
)

In [ ]:
# Preprocessing pipeline for selected features
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer

num_selected = [col for col in selected_features if col in num_cols]
cat_selected = [col for col in selected_features if col in cat_cols]

numeric_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", RobustScaler())
])
categorical_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])
preprocessor = ColumnTransformer([
    ("num", numeric_pipe, num_selected),
    ("cat", categorical_pipe, cat_selected)
])

In [ ]:
# Encode target for SMOTE
le_y = LabelEncoder()
y_train_cls_enc = le_y.fit_transform(y_train_cls)

# Preprocess X_train for SMOTE (must be numeric)
X_train_for_smote = preprocessor.fit_transform(X_train_cls)
# Convert to DataFrame for SMOTE compatibility
import pandas as pd
X_train_for_smote = pd.DataFrame(X_train_for_smote)

# Balance classes with SMOTE
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_cls_bal, y_train_cls_bal = smote.fit_resample(X_train_for_smote, y_train_cls_enc)

# Prepare X_test for evaluation
X_test_for_eval = preprocessor.transform(X_test_cls)

In [ ]:
classifiers = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42),
    "CatBoost": CatBoostClassifier(verbose=0, random_state=42),
    "MLPClassifier": MLPClassifier(max_iter=500, random_state=42)
}

In [ ]:
for name, model in classifiers.items():
    # Fit on balanced data
    model.fit(X_train_cls_bal, y_train_cls_bal)
    # Predict on test set (need to encode y_test for report)
    y_pred = model.predict(X_test_for_eval)
    y_test_cls_enc = le_y.transform(y_test_cls)
    print(f"\n{name}")
    print(classification_report(y_test_cls_enc, y_pred, target_names=le_y.classes_))
    print("Accuracy:", accuracy_score(y_test_cls_enc, y_pred))

In [ ]:
from sklearn.model_selection import GridSearchCV

# Encode target labels as integers for classification
le_y = LabelEncoder()
y_train_cls_enc = le_y.fit_transform(y_train_cls)
y_test_cls_enc = le_y.transform(y_test_cls)

# Define parameter grids for each model
param_grids = {
    "Random Forest": {
        "model__n_estimators": [100, 200],
        "model__max_depth": [None, 5, 10]
    },
    "Logistic Regression": {
        "model__C": [0.1, 1, 10],
        "model__solver": ["lbfgs", "liblinear"]
    },
    "XGBoost": {
        "model__n_estimators": [100, 200],
        "model__max_depth": [3, 5, 7]
    },
    "CatBoost": {
        "model__iterations": [100, 200],
        "model__depth": [4, 6, 8]
    },
    "MLPClassifier": {
        "model__hidden_layer_sizes": [(50,), (100,)],
        "model__alpha": [0.0001, 0.001]
    }
}

best_models = {}
for name, model in classifiers.items():
    pipe = Pipeline([
        ("preprocessor", preprocessor),
        ("model", model)
    ])
    param_grid = param_grids.get(name, {})
    grid = GridSearchCV(pipe, param_grid, scoring='f1_weighted', cv=3, n_jobs=-1)
    grid.fit(X_train_cls, y_train_cls_enc)
    best_models[name] = grid.best_estimator_
    y_pred = best_models[name].predict(X_test_cls)
    print(f"\n{name} (Best Params: {grid.best_params_})")
    print(classification_report(y_test_cls_enc, y_pred, target_names=le_y.classes_))
    print("Accuracy:", accuracy_score(y_test_cls_enc, y_pred))

In [ ]:
# Confusion matrix display and save models
from sklearn.metrics import ConfusionMatrixDisplay
for name, model in best_models.items():
    y_pred = model.predict(X_test_cls)
    plt.figure(figsize=(5, 5))
    ConfusionMatrixDisplay.from_predictions(
        y_test_cls_enc, y_pred, display_labels=le_y.classes_
    )
    plt.title(f"{name} Best Model Confusion Matrix")
    plt.show()

In [ ]:
os.makedirs("models/saved_models", exist_ok=True)
os.makedirs("reports/figures", exist_ok=True)

# Save classification models
for name, model in best_models.items():
    file_path = f"models/saved_models/{name.replace(' ', '_').lower()}_best_classifier.joblib"
    joblib.dump(model, file_path)
    print(f"Saved {file_path}")

In [ ]:
# SHAP Feature Importance for all best classification models
for name, model in best_models.items():
    # Get the model from the pipeline
    clf = model.named_steps["model"]
    # Get preprocessed test data
    X_test_transformed = model.named_steps["preprocessor"].transform(X_test_cls)
    # SHAP expects a model, not a pipeline
    try:
        explainer = shap.Explainer(clf, X_test_transformed)
        shap_values = explainer(X_test_transformed)
        plt.figure()
        shap.summary_plot(shap_values, X_test_transformed, feature_names=model.named_steps["preprocessor"].get_feature_names_out())
        plt.title(f"SHAP Summary for {name}")
        plt.show()
    except Exception as e:
        print(f"SHAP not supported for {name}: {e}")

In [ ]:
# Save SHAP plots for classifiers
for name, model in best_models.items():
    try:
        clf = model.named_steps["model"]
        X_test_transformed = model.named_steps["preprocessor"].transform(X_test_cls)
        explainer = shap.Explainer(clf, X_test_transformed)
        shap_values = explainer(X_test_transformed)
        shap.summary_plot(shap_values, X_test_transformed, 
                          feature_names=model.named_steps["preprocessor"].get_feature_names_out(), 
                          show=False)
        plt.title(f"SHAP Summary: {name}")
        plt.savefig(f"reports/figures/shap_{name.replace(' ', '_').lower()}_cls.png")
        plt.close()
    except Exception as e:
        print(f"SHAP not supported for {name}: {e}")